# Логистическая регрессия, метод опорных векторов, one-hot кодирование

### О задании

В этом задании вы:
- настроите метод опорных векторов
- изучите методы работы с категориальными переменными

In [1]:
import pandas as pd
import numpy as np
from time import time

from sklearn.base import BaseEstimator
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

__Задание 1.__ Обучение логистической регрессии на реальных данных и оценка качества классификации.

**(5 баллов)**


Загрузим данные с конкурса [Kaggle Porto Seguro’s Safe Driver Prediction](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction) (вам нужна только обучающая выборка). Задача состоит в определении водителей, которые в ближайший год воспользуются своей автомобильной страховкой (бинарная классификация). Но для нас важна будет не сама задача, а только её данные. При этом под нужды задания мы немного модифицируем датасет.

In [2]:
import pandas as pd
data = pd.read_csv('train.csv', index_col=0)
target = data.target.values
data = data.drop('target', axis=1)

Пересемплируем выборку так, чтобы положительных и отрицательных объектов в выборке было одинаковое число. Разделим на обучающую и тестовую выборки.

In [3]:
from sklearn.model_selection import train_test_split
np.random.seed(910)
mask_plus = np.random.choice(np.where(target == 1)[0], 100000, replace=True)
mask_zero = np.random.choice(np.where(target == 0)[0], 100000, replace=True)

data = pd.concat((data.iloc[mask_plus], data.iloc[mask_zero]))
target = np.hstack((target[mask_plus], target[mask_zero]))

Не забудьте отнормировать признаки (можно воспользоваться StandardScaler или сделать это вручную). Пока не будем обращать внимание на то, что некоторые признаки категориальные (этим мы займёмся позже).

In [4]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.5)
X_train, X_test = StandardScaler().fit_transform(X_train), StandardScaler().fit_transform(X_test)

Обучите логистическую регрессию с удобными для вас параметрами, примените регуляризацию. Сделайте предсказание на тестовой части выборки. Посчитайте accuracy, precision, recall и F меру

In [5]:
def accuracy(y_pred, y_real):
    return len(y_real[y_pred == y_real]) / len(y_real)

def precision(y_pred, y_real):
    return len(y_real[(y_pred == 1) & (y_real == y_pred)]) / len(y_real[y_pred == 1])

def recall(y_pred, y_real):
    return len(y_real[(y_real == 1) & (y_real == y_pred)]) / len(y_real[y_real == 1])

In [6]:
from sklearn.linear_model import LogisticRegression

reg = LogisticRegression(C=1)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
xac = accuracy(y_pred, y_test)
xpr = precision(y_pred, y_test)
xrec = recall(y_pred, y_test)
xf = 2 * xpr * xrec / (xpr + xrec)

print(f"Accuracy: {xac}\nPrecision: {xpr}\nRecall: {xrec}\nF: {xf}")

Accuracy: 0.5882
Precision: 0.5958472698840189
Recall: 0.548614166300044
F: 0.5712560386473431


__Выводы__ в свободной форме: логистическая регрессия дает примерно одинаковый результат на всех метриках ($\approx 0.6$)

## Часть 2. Работа с категориальными переменными

В этой части мы научимся обрабатывать категориальные переменные, так как закодировать их в виде чисел недостаточно (это задаёт некоторый порядок, которого на категориальных переменных может и не быть). Существует два основных способа обработки категориальных значений:
- One-hot-кодирование
- Счётчики (CTR, mean-target кодирование, ...) — каждый категориальный признак заменяется на среднее значение целевой переменной по всем объектам, имеющим одинаковое значение в этом признаке.

Начнём с one-hot-кодирования. Допустим наш категориальный признак $f_j(x)$ принимает значения из множества $C=\{c_1, \dots, c_m\}$. Заменим его на $m$ бинарных признаков $b_1(x), \dots, b_m(x)$, каждый из которых является индикатором одного из возможных категориальных значений:
$$
b_i(x) = [f_j(x) = c_i]
$$

__Задание 1.__ Закодируйте все категориальные признаки с помощью one-hot-кодирования. Обучите логистическую регрессию и посмотрите, как изменилось качество модели (с тем, что было ранее). Измерьте время, потребовавшееся на обучение модели.

__(3 балла)__

In [7]:
columns = data.columns

cat_columns = []

for col in columns:
    if col[-3:] == 'cat':
        cat_columns.append(col)

In [8]:
data_onehot = np.hstack([OneHotEncoder().fit_transform(data[cat_columns]).toarray(), data.drop(columns=cat_columns).to_numpy()])

X_train, X_test, y_train, y_test = train_test_split(data_onehot, target, test_size=0.5)

In [9]:
reg = LogisticRegression(C=1, max_iter=1000)

st = time()
reg.fit(X_train, y_train)
print(f"Learning time: {time() - st}")

y_pred = reg.predict(X_test)
xac = accuracy(y_pred, y_test)
xpr = precision(y_pred, y_test)
xrec = recall(y_pred, y_test)
xf = 2 * xpr * xrec / (xpr + xrec)

print(f"Accuracy: {xac}\nPrecision: {xpr}\nRecall: {xrec}\nF: {xf}")

Learning time: 66.05493187904358
Accuracy: 0.5961
Precision: 0.6027069212512561
Recall: 0.5637938276296578
F: 0.5826013269123452


Как можно было заменить, one-hot-кодирование может сильно увеличивать количество признаков в датасете, что сказывается на памяти, особенно, если некоторый признак имеет большое количество значений. Эту проблему решает другой способ кодирование категориальных признаков — счётчики. Основная идея в том, что нам важны не сами категории, а значения целевой переменной, которые имеют объекты этой категории. Каждый категориальный признак мы заменим средним значением целевой переменной по всем объектам этой же категории:
$$
g_j(x, X) = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)][y_i = +1]}{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}
$$

__Задание 2.__ Закодируйте категориальные переменные с помощью счётчиков (ровно так, как описано выше без каких-либо хитростей). Обучите логистическую регрессию и посмотрите на качество модели на тестовом множестве. Сравните время обучения с предыдущим экспериментов. Заметили ли вы что-то интересное?

__(2 балла)__

In [10]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.5)
train = X_train
test = X_test

for column in cat_columns:
    values = data[column].unique()
    for val in values:
        mean = y_train[X_train[column] == val].mean()
        train[column][X_train[column] == val] = mean
        test[column][X_test[column] == val] = mean

/tmp/ipykernel_6417/504736265.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[column][X_train[column] == val] = mean
/tmp/ipykernel_6417/504736265.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[column][X_test[column] == val] = mean
/tmp/ipykernel_6417/504736265.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[column][X_train[column] == val] = mean
/tmp/ipykernel_6417/504736265.py:10: SettingWithCopyWarning

In [11]:
reg = LogisticRegression(C=1, max_iter=1000)

st = time()
reg.fit(train, y_train)
print(f"Learning time: {time() - st}")

y_pred = reg.predict(test)
xac = accuracy(y_pred, y_test)
xpr = precision(y_pred, y_test)
xrec = recall(y_pred, y_test)
xf = 2 * xpr * xrec / (xpr + xrec)

print(f"Accuracy: {xac}\nPrecision: {xpr}\nRecall: {xrec}\nF: {xf}")

Learning time: 42.77199697494507
Accuracy: 0.59271
Precision: 0.5959165779420054
Recall: 0.5718378140149846
F: 0.5836289473415185


/home/semen/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


__Вывод:__ счётчики не дают существенного выигрыша в точности по сравнению с one-hot, однако существенно уменьшают затрачиваемую память и время работы. Оба метода дают лучшую точность, чем нормализация категориальных факторов

Отметим, что такие признаки сами по себе являются классификаторами и, обучаясь на них, мы допускаем "утечку" целевой переменной в признаки. Это ведёт к переобучению, поэтому считать такие признаки необходимо таким образом, чтобы при вычислении для конкретного объекта его целевая метка не использовалась. Это можно делать следующими способами:
- вычислять значение счётчика по всем объектам расположенным выше в датасете (например, если у нас выборка отсортирована по времени)
- вычислять по фолдам, то есть делить выборку на некоторое количество частей и подсчитывать значение признаков по всем фолдам кроме текущего (как делается в кросс-валидации)
- внесение некоторого шума в посчитанные признаки (необходимо соблюсти баланс между избавление от переобучения и полезностью признаков).

__Задание 3.__ Реализуйте корректное вычисление счётчиков двумя из трех вышеперчисленных способов, сравните. Снова обучите логистическую регрессию, оцените качество. Сделайте выводы.

__(3 балла)__

Метод 3. Добавление шума

In [34]:
folds_cnt = int(np.ceil(len(data) ** 0.5))
data_mod = data.copy()

d = {}
dall = {}

for column in cat_columns:
    for val in data[column].unique():
        d[(column, val)] = [[0, 0] for _ in range(folds_cnt)]

for i in range(folds_cnt):
    mn = i * folds_cnt
    mx = (i + 1) * folds_cnt

    dt = data.iloc[mn:mx]
    tr = target[mn:mx]

    for column in cat_columns:
        for val in data[column].unique():
            c = len(tr[dt[column] == val])
            v = tr[dt[column] == val].sum()
            d[(column, val)][i] = [c, v]

for column in cat_columns:
    for val in data[column].unique():
        dall[(column, val)] = [0, 0]
        for i in range(folds_cnt):
            dall[(column, val)][0] += d[(column, val)][i][0]
            dall[(column, val)][1] += d[(column, val)][i][1]

In [36]:
for i in range(folds_cnt):
    mn = i * folds_cnt
    mx = (i + 1) * folds_cnt

    for column in cat_columns:
        for val in data[column].unique():
            mean = (dall[(column, val)][1] - d[(column, val)][i][1]) / (dall[(column, val)][0] - d[(column, val)][i][0])
            data_mod.iloc[mn:mx][column][data.iloc[mn:mx][column] == val] = pd.Series(mean + np.random.uniform(-0.05, 0.05, len(data.iloc[mn:mx][column][data.iloc[mn:mx][column] == val])))


/tmp/ipykernel_6417/3729023326.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_mod.iloc[mn:mx][column][data.iloc[mn:mx][column] == val] = pd.Series(mean + np.random.uniform(-0.05, 0.05, len(data.iloc[mn:mx][column][data.iloc[mn:mx][column] == val])))
/tmp/ipykernel_6417/3729023326.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_mod.iloc[mn:mx][column][data.iloc[mn:mx][column] == val] = pd.Series(mean + np.random.uniform(-0.05, 0.05, len(data.iloc[mn:mx][column][data.iloc[mn:mx][column] == val])))
/tmp/ipykernel_6417/3729023326.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [37]:
train, test, y_train, y_test = train_test_split(data_mod, target, test_size=0.5)

In [38]:
reg = LogisticRegression(C=1, max_iter=1000)

st = time()
reg.fit(train, y_train)
print(f"Learning time: {time() - st}")

y_pred = reg.predict(test)
xac = accuracy(y_pred, y_test)
xpr = precision(y_pred, y_test)
xrec = recall(y_pred, y_test)
xf = 2 * xpr * xrec / (xpr + xrec)

print(f"Accuracy: {xac}\nPrecision: {xpr}\nRecall: {xrec}\nF: {xf}")

Learning time: 45.359808921813965
Accuracy: 0.58878
Precision: 0.5946750668853025
Recall: 0.5523668283297928
F: 0.5727406853271825


/home/semen/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


__Вывод:__ в данном примере оптимизации счётчиков не дают значительного изменения.

## Часть 3. Метод опорных векторов и калибровка вероятностней

__Задание 1.__ Обучение и применение метода опорных векторов.

__(1 балл)__

Обучите метод опорных векторов (воспользуйтесь готовой реализацией LinearSVC из sklearn). Используйте уже загруженные и обработанные в предыдущей части данные.

In [24]:
from sklearn.svm import LinearSVC

svc = LinearSVC(random_state=0)

svc.fit(train, y_train)

/home/semen/.local/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/semen/.local/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(random_state=0)

На той же тестовой части посчитайте все те же метрики. Что вы можете сказать о полученных результатах?

In [25]:
y_pred = svc.predict(test)
xac = accuracy(y_pred, y_test)
xpr = precision(y_pred, y_test)
xrec = recall(y_pred, y_test)
xf = 2 * xpr * xrec / (xpr + xrec)

print(f"Accuracy: {xac}\nPrecision: {xpr}\nRecall: {xrec}\nF: {xf}")

Accuracy: 0.50672
Precision: 0.5031044502217464
Recall: 0.9881851132426858
F: 0.6667522395319615


__Вывод:__ метод опорных векторов улучшает recall и f-меру, незначительно ухудшая при этом остальные метрики